MIT License

Copyright (c) 2024 Mohannad Shehadeh

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Higher-Order Staircase Code Interleaver Maps

This [Julia](https://julialang.org/) language notebook constructs encoders for strictly memory-optimal [Higher-Order Staircase Codes](https://arxiv.org/pdf/2312.13415) in the format of
of [Zipper Code](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9843869) interleaver maps. It also generates the corresponding [ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html) configuration files as well as visualizations of the encoding procedure.

Start by installing the required packages by uncommenting as needed.

In [1]:
# import Pkg; Pkg.add("Nemo")
using Nemo
# import Pkg; Pkg.add("DelimitedFiles")
using DelimitedFiles
include("L-one-two-DTSs.jl");


Welcome to Nemo version 0.45.3

Nemo comes with absolutely no warranty whatsoever


Choose a degree parameter $M\in \{1,2,\dots,9\}$. The bit degree will be $M+1$.

In [2]:
M = 2

2

Choose a tile size $T$ which satisfies one of the following conditions:
- $T = 1$
- $T$ is a prime power and $M \leq T$
- $M \leq \text{ the least prime factor of } T$

Note that $T = S/L$ in the [Higher-Order Staircase Codes](https://arxiv.org/pdf/2312.13415) paper.

In [3]:
T = 2

2

In [4]:
factors = factor(T)
prime_factors = [x[1] for x in factors];
prime_factor_powers = [x[2] for x in factors];
@assert prod(prime_factors .^ prime_factor_powers) == T
if T == 1
    net_type = "trivial"
elseif length(prime_factors) == 1 && prime_factor_powers[1] > 1
    p = prime_factors[1]
    k = prime_factor_powers[1]
    @assert T == p^k
    F_T, α = finite_field(p,k,"α")
    # α is a primitive element of the finite field F_T
    @assert is_gen(α) 
    # elements of a finite field of size T: 0,1,α,α^2,...,α^(T-2) (can do finite field arithmetic on)
    F_T_SET = [i == -1 ? 0α : α^i for i in -1:T-2]
    # dictionary mapping 0,1,α,α^2,...,α^(T-2) to 0,1,...,T-1
    F_T_LABELS = Dict(F_T_SET[i] => i-1 for i in 1:T) 
    @assert [F_T_LABELS[F_T_SET[i]] for i in 1:T] == 0:T-1
    net_type = "prime power"
elseif M <= minimum(prime_factors)
    net_type = "sufficient lpf" 
else
    throw("Tile size requirements not satisfied!")
end

"sufficient lpf"

If $T \neq 1$, we must define $M$ intra-tile permutations (excluding the identity permutation). We can choose whether or not we want the intra-tile permutations to be involutions (meaning that they are equal to their own inverses) by setting $\textsf{involutions}$ to be $\textsf{true}$ or $\textsf{false}$.

If $\textsf{false}$, the intra-tile permutations are defined by multiplying indices
$\begin{pmatrix} i & j\end{pmatrix}$ from the right by matrices of the form
$$
\begin{pmatrix}
 0 & 1\\
 1 & z
\end{pmatrix}
$$
which corresponds to transposing the tile and then permuting its columns. The inverse of this matrix is 
$$
\begin{pmatrix}
 -z & 1\\
 1 & 0
\end{pmatrix}\text{.}
$$
If $\textsf{true}$, then the intra-tile permutations are defined by multiplying indices
$\begin{pmatrix} i & j\end{pmatrix}$ from the right by matrices of the form
$$
\begin{pmatrix}
 -z & 1 - z^2 \\
 1 & z
\end{pmatrix}
$$
which are equal to their own inverses meaning that all intra-tile permutations can be undone by permuting again according to the same permutation. 

There is no theoretical reason to prefer one over the other so this can be chosen based on what is most practically convenient. The first option has the advantage of being amenable to hardware architectures that have already been designed around matrix transposition such as that of [Truhachev et al.](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9228874&tag=1). The second option has the advantage of saving you from the confusion about when to apply the permutation or when to apply its inverse. Such a mistake is not harmless: It is absolutely crucial that the forward and inverse permutations are not swapped in the code construction **even if done consistently**.

If $T$ is a prime power, then we consider a finite field $\mathbb{F}_T = \{0,1,\alpha,\alpha^2,\dots,\alpha^{T-2}\}$ and perform permutations by $\mathbb{F}_T$-arithmetic with $z \in \{0,1,\alpha,\dots,\alpha^{M-2}\}$ and $M\leq T$.

Otherwise, we perform modulo-$T$ arithmetic with $z \in \{0,1,\dots,M-1\}$ and $M \leq \text{ the least prime factor of } T$.

We represent the permutations by $T \times T \times M$ lookup tables. In practice, if $T$ is large, it's probably to better to compute them on the fly by implementing the modular or finite-field arithmetic as needed.

In [5]:
involutions = false

false

In [6]:
if net_type == "prime power"
    if involutions == false
        π_LUT = [(F_T_LABELS[F_T_SET[j]], F_T_LABELS[F_T_SET[i]+F_T_SET[k]*F_T_SET[j]]) for i in 1:T, j in 1:T, k in 1:M]
        π_inv_LUT = [(F_T_LABELS[-F_T_SET[k]*F_T_SET[i]+F_T_SET[j]], F_T_LABELS[F_T_SET[i]]) for i in 1:T, j in 1:T, k in 1:M]
    else # involutions
        π_LUT = [(F_T_LABELS[-F_T_SET[k]*F_T_SET[i]+F_T_SET[j]], F_T_LABELS[F_T_SET[i]-F_T_SET[k]^2*F_T_SET[i]+F_T_SET[k]*F_T_SET[j]]) for i in 1:T, j in 1:T, k in 1:M]
        π_inv_LUT = copy(π_LUT)
    end
    @assert [π_inv_LUT[π_LUT[i,j,k][1]+1, π_LUT[i,j,k][2]+1, k] .+ 1 for i in 1:T, j in 1:T, k in 1:M] == [(i,j) for i in 1:T, j in 1:T, k in 1:M]
    @assert [π_LUT[π_inv_LUT[i,j,k][1]+1, π_inv_LUT[i,j,k][2]+1, k] .+ 1 for i in 1:T, j in 1:T, k in 1:M] == [(i,j) for i in 1:T, j in 1:T, k in 1:M]
elseif net_type == "sufficient lpf"
    if involutions == false
        π_LUT = [mod.( (j, i + z*j) , T)  for i in 0:T-1, j in 0:T-1, z in 0:M-1]
        π_inv_LUT = [mod.( (-z*i + j, i) , T) for i in 0:T-1, j in 0:T-1, z in 0:M-1]
    else # involutions
        π_LUT = [mod.( (-z*i + j, (1-z^2)*i + z*j) , T) for i in 0:T-1, j in 0:T-1, z in 0:M-1]
        π_inv_LUT = copy(π_LUT)
    end
    @assert [π_inv_LUT[π_LUT[i,j,k][1]+1, π_LUT[i,j,k][2]+1, k] .+ 1 for i in 1:T, j in 1:T, k in 1:M] == [(i,j) for i in 1:T, j in 1:T, k in 1:M]
    @assert [π_LUT[π_inv_LUT[i,j,k][1]+1, π_inv_LUT[i,j,k][2]+1, k] .+ 1 for i in 1:T, j in 1:T, k in 1:M] == [(i,j) for i in 1:T, j in 1:T, k in 1:M]
end

The following cell is optional and visualizes the permutations by applying them to the first matrix displayed. These permutations have the property that any row of any matrix intersects with a row of any other matrix in precisely one position. The number-theoretic conditions on $T$ are needed to be able to construct such permutations.

In [7]:
if T != 1
    tile = [j + T*i + 1 for i in 0:T-1, j in 0:T-1];
    display(tile)
    for k in 1:M
        display([tile[π_LUT[i,j,k][1] + 1,π_LUT[i,j,k][2] + 1] for i in 1:T, j in 1:T])
    end
end

2×2 Matrix{Int64}:
 1  2
 3  4

2×2 Matrix{Int64}:
 1  3
 2  4

2×2 Matrix{Int64}:
 1  4
 2  3


Choose a value of $L$ such that your component code length is
$(M+1)LT$ and the number of parity bits of your component code (length minus dimension) is $r \leq LT$. This determines the overall rate of your higher-order staircase code:
$$
R_\mathsf{unterminated} = 1-\frac{r}{LT}
$$

Currently, the supported values of $L$ are:
- When $M = 1$: all $L\geq 1$
- When $M = 2$: all $L\geq 1$
- When $M = 3$: $L \in \{1,2,\dots,15\}$
- When $M = 4$: $L \in \{1,2,\dots,8,10\}$
- When $M \in \{5,6,7,8,9\}$: $L = 1$

Note that the value of $L$ should theoretically have a relatively small impact on code performance. Therefore, for a constant code rate determined by constant $LT$, we can choose various $L$ and $T$ subject to the current constraints on $L$ and $T$. A larger $L$ results in encoding and decoding memory reductions by up to the following theoretical limits in the regime of high code rates:

| M  | encoding | decoding |
|---|---|---|
| 1  | 50.00%  | 50.00%  |
| 2  | 25.00%  | 25.00%  |
| 3  | 16.67%  | 14.29%  |
| 4  | 18.18%  | 16.67%  |

Note further that the restrictions on the values of $L$ above are simply because those are the values for which we have found or know of special combinatorial objects (sum-of-lengths-optimal difference triangle sets) which translate to memory-optimal higher-order staircase codes approaching the above limits. These can be expanded in the future.

Lastly, note that $T$ also determines the number of component code encoders that can naturally operate in parallel during the encoding procedure. This also informs the choice of $L$ for a given code rate. 

Therefore, $L$ can be regarded as a memory–parallelism tradeoff parameter with both extremes $T = 1$ and $L = 1$
leading to nontrivial codes which support any code rate and bit degree (subject to current limitations on $L$).

In [8]:
L = 2

2

In [9]:
if M == 1
    DTS = trivial_dts(L);
elseif M == 2;
    DTS = skokeefe(L);
elseif M == 3 || M == 4
    DTS_name_list = readdir("sum-of-lengths-scope-optimal-DTSs/");
    DTS_index = findlast(startswith.(DTS_name_list, string(L)*"-"*string(M)));
    if DTS_index == nothing
        throw("L and M combination is currently unsupported!")
    else
        DTS = readdlm("sum-of-lengths-scope-optimal-DTSs/"*DTS_name_list[DTS_index], Int64);
    end 
elseif L == 1 && M == 5
    DTS = [0 1 4 10 12 17]
elseif L == 1 && M == 6
    DTS = [0 1 4 10 18 23 25]
elseif L == 1 && M == 7
    DTS = [0 1 4 9 15 22 32 34]
elseif L == 1 && M == 8
    DTS = [0 1 5 12 25 27 35 41 44]
elseif L == 1 && M == 9
    DTS = [0 1 6 10 23 26 34 41 53 55]
else
    throw("L and M combination is currently unsupported!")
end

2×3 Matrix{Int64}:
 0  2  7
 0  3  4

The difference triangle set (DTS) determined above determines inter-tile delays such that distinct component codewords share at most a single tile. We have a choice in how to order the tiles in constructing the code by setting $\textsf{naturalDelays}$ to $\textsf{true}$ or $\textsf{false}$. Arbitrary orderings of these delays beyond the two options I provide are admissible. The convention I use in [Higher-Order Staircase Codes](https://arxiv.org/pdf/2312.13415) is $\textsf{naturalDelays} = \textsf{true}$. You can visualize them and choose whatever is more natural to you or convenient for the implementation method you have in mind. 

In [10]:
naturalDelays = false

false

A codeword (or codeword stream) of a higher-order staircase code can be represented as $L$ streams of $T\times T$ tiles. The streams are subjected to inter-tile delays and intra-tile permutations. The following triples $(a,b,c)$ consist of the stream label $a \in \{0,1,\dots,L-1\}$, the permutation index $b \in \{1,2,\dots,M\}$, and the tile delay $c$.

In [11]:
block_delays = vec([(i-1, j-1, DTS[i,j]) for i in 1:L, j in 2:M+1])

4-element Vector{Tuple{Int64, Int64, Int64}}:
 (0, 1, 2)
 (1, 1, 3)
 (0, 2, 7)
 (1, 2, 4)

In [12]:
if naturalDelays == true
    block_delays = sort(block_delays, by = x -> x[3], rev = true)
else
    block_delays = sort(block_delays, by = x -> (x[2],x[1]), rev = true)
end

4-element Vector{Tuple{Int64, Int64, Int64}}:
 (1, 2, 4)
 (0, 2, 7)
 (1, 1, 3)
 (0, 1, 2)

In [13]:
# block_delays = reverse(block_delays)

We now translate higher-order staircase codes into the language of [Zipper Codes](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9843869). The transmitted codeword (or codeword stream) is represented by a semi-infinite matrix of width $LT$ referred to as the *real buffer*:
$$
\begin{pmatrix}
\vdots\\
\text{——}\; \mathbf{0} \; \text{——} \\
\text{——} \; \mathbf{0} \; \text{——}  \\
\text{——} \; \mathbf{c}_{1} \; \text{——}  \\
\text{——} \; \mathbf{c}_{2} \; \text{——}  \\
\text{——} \; \mathbf{c}_{3} \; \text{——} \\
\vdots
\end{pmatrix}
$$
Each $\mathbf{c}_i$ consists of $LT-r$ information bits and $r$ parity bits. A *virtual buffer* of width $MLT$ is adjoined
to the real buffer to form $\mathsf{BUFFER}$ where
$$
\mathsf{BUFFER} =
\begin{pmatrix}
\vdots &\vdots\\
\text{——}\; \mathbf{0} \; \text{——}  &\text{——}\; \mathbf{0} \; \text{——} \\
\text{——}\; \mathbf{0} \; \text{——}  &\text{——} \; \mathbf{0} \; \text{——}  \\
\text{——} \; \mathbf{v}_{1} \; \text{——}  &\text{——} \; \mathbf{c}_{1} \; \text{——}  \\
\text{——} \; \mathbf{v}_{2} \; \text{——}  &\text{——} \; \mathbf{c}_{2} \; \text{——}  \\
\text{——} \; \mathbf{v}_{3} \; \text{——}  &\text{——} \; \mathbf{c}_{3} \; \text{——} \\
\vdots &\vdots
\end{pmatrix}\text{.}
$$
Each $\mathbf{v}_i$ contains $MLT$ bits that are *copied* from the real buffer thus do not need to be transmitted; only the real buffer is transmitted. Rows
of $\mathsf{BUFFER}$ belong to some specified component code of length $(M+1)LT$ and dimension $(M+1)LT-r$. This
is achieved by specifying the entries of $\mathbf{v}_i$ which are given by $(\mathsf{BUFFER})_{(i,j)}$ for 
$j\in \{0,1,\dots, MLT - 1\}$ as 
$$
(\mathsf{BUFFER})_{(i,j)} = (\mathsf{BUFFER})_{\phi(i,j)}
$$
where the function $\phi(i,j)$:
\begin{align*}
\phi \colon \mathbb{Z}\times \{0,1,\dots, MLT - 1\} &\longrightarrow \mathbb{Z} \times \{MLT+0, MLT+1,\dots, MLT + LT -1\}\\
                        (i,j) &\longmapsto \phi(i,j) = (\phi_1(i,j),\phi_2(i,j))
\end{align*}
is termed the *interleaver map*. By definition, the interleaver map only copies from the real buffer. To practically realize an encoding procedure, we need causality, i.e., that $\phi_1(i,j) \leq i$. In addition to causality, we usually need periodicity meaning that there 
exists $\nu$ such that $\phi(i + \nu,j) = \phi(i,j) + (\nu, 0)$. For higher-order staircase codes, we have causality and 
period $\nu = T$. A periodic interleaver map is fully specified by its values over one period $\phi(i,j)$ for 
$i \in \{0,1,\dots,T-1\}$ and $j\in \{0,1,\dots,MLT-1\}$.

The following function defines the interleaver map for the [Higher-Order Staircase Code](https://arxiv.org/pdf/2312.13415) for which we have at this point specified all parameters. 

In [14]:
if T != 1
    function ϕ(i,j)
        (input_class, permutation_index, block_delay_value) = block_delays[floor(Int64,j/T) + 1];
        ϕ_1 = floor(Int64,i/T)*T - block_delay_value*T + π_LUT[mod(i,T) + 1, mod(j,T) + 1, permutation_index][1]
        ϕ_2 = M*L*T + (L-1-input_class)*T + π_LUT[mod(i,T) + 1, mod(j,T) + 1, permutation_index][2]
        return (ϕ_1, ϕ_2)
    end;
else
    function ϕ(i,j)
        (input_class, permutation_index, block_delay_value) = block_delays[j + 1];
        ϕ_1 = i - block_delay_value
        ϕ_2 = M*L + (L-1-input_class)
        return (ϕ_1, ϕ_2)
    end;
end

ϕ (generic function with 1 method)

The following cell is optional and produces a visualization of the encoding pattern in the form of a text file representing $\mathsf{BUFFER}$. 
We represent $\mathbf{c}_1$ as $1,2,\dots,LT$, $\mathbf{c}_2$ as $LT + 1, LT + 2,\dots, 2LT$, and so on. This cell should only be
run with small values of $T$ to avoid writing a very large text file and possibly crashing your browser. In this visualization,
no pair of integers should ever appear in the same row twice. This is a key property of [classical staircase codes](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=6074908) that is preserved
by [Higher-Order Staircase Codes](https://arxiv.org/pdf/2312.13415).

In [15]:
numRows = T*maximum(DTS) + 3*T
virtual_real = zeros(Int64, T*maximum(DTS), (M+1)*L*T)
offset = size(virtual_real)[1];
for row in 0:numRows-1
    virtual = [virtual_real[ϕ(row+offset, j)[1] + 1, ϕ(row+offset, j)[2] + 1] for j in 0:M*L*T-1]'
    real = [j + L*T*row + 1 for j in 0:L*T-1]'
    virtual_real = [virtual_real; [virtual real]]
end
writedlm("virtual_real.txt", virtual_real)

In [16]:
;cat virtual_real.txt

0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	0	0	0	0
0	0	0	0	0	0	0	0	1	2	3	4
0	0	0	0	0	0	0	0	5	6	7	8
0	0	0	0	0	0	0	0	9	10	11	12
0	0	0	0	0	0	0	0	13	14	15	16
0	0	0	0	0	0	3	7	17	18	19	20
0	0	0	0	0	0	4	8	21	22	23	24
0	0	0	0	1	5	11	15	25	26	27	28
0	0	0	0	2	6	12	16	29	30	31	32
1	6	0	0	9	13	19	23	33	34	35	36
2	5	0	0	10	14	20	24	37	38	39	40
9	14	0	0	17	21	27	31	41	42	43	44
10	13	0	0	18	22	28	32	45	46	47	48
17	22	0	0	25	29	35	39	49	50	51	52
18	21	0	0	26	30	36	40	53	54	55	56
25	30	3	8	33	37	43	47	57	58	59	60
26	29	4	7	34	38	44	48	61	62	63	64
33	38	11	16	41	45	51	55	65	66	67	68
34	37	12	15	42	46	52	56	69	70	71	72
41	46	19	24	49	53	59	63	73	74	75	76
42	45	20	23	50	54	60	64	77	78	79	80


We now now produce a [ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html) configuration file corresponding to our interleaver map. [ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html) currently only supports triple-error-correcting BCH codes with lengths $n \in \{127,255,511,1023\}$ and corresponding numbers of parity bits $r\in \{21,24,27,30\}$ as component codes (as well as component codes shortened from these). 
We must therefore have a component code length $(M+1)LT \leq n$ for some $n$ as just listed. We must further have $r \leq LT$. Note further that ZipperSim requires that the parent BCH code length $n$ be specified directly in the code by commenting and uncommenting in the source code as needed.

Choose the parent component code length $n$:

In [ ]:
n = 127

In [ ]:
if n ∉ [127,255,511,1023]
    throw("Parent code length unsupported by ZipperSim!")
else
    if n == 127
        r = 21
    elseif n == 255
        r = 24
    elseif n == 511
        r = 27
    elseif n == 1023
        r = 30
    end
end

In [ ]:
if (M+1)*L*T > n
    throw("Parent component code too short!")
end
if r > L*T
    throw("r is too large!")
end

In [ ]:
println("virtual buffer width (m in ZipperSim): $(M*L*T) ")
println("                    real buffer width: $(L*T) ")
println("                component code length: $((M+1)L*T) ")
println("     higher-order staircase code rate: $(1-r/(L*T)) ")

[ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html) measures the decoding window size by the number of component codewords in the decoding window, or, equivalently, the number of rows of the real buffer in the decoding window. This quantity is termed $\textsf{bufferLen}$. As an empirically-determined rule of thumb, the decoding window size should be $2$ to $5$ times the constraint length defined in [Higher-Order Staircase Codes](https://arxiv.org/pdf/2312.13415). For [ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html), the iteration count is specified in the source code while the decoding window size is specified in the configuration file.

Choose a constraint-length-multiple to determine the decoding window size:

In [ ]:
cl_multiple = 3 

In [ ]:
bufferLen = cl_multiple*(1+maximum(DTS))*T

The decoding window size in megabits is as follows:

In [ ]:
bufferLen*L*T/(1e6)

The following cell will create the [ZipperSim](https://www.comm.utoronto.ca/~frank/ZipperSim/index.html) configuration file:

In [ ]:
ZS_config = ["period $T"]
ZS_config = vcat(ZS_config, "n"* prod(" " .* string.(ones(Int64, 1, T)*(M+1)*L*T)))
ZS_config = vcat(ZS_config, "k"* prod(" " .* string.(ones(Int64, 1, T)*((M+1)*L*T-r))))
ZS_config = vcat(ZS_config, "m"* prod(" " .* string.(ones(Int64, 1, T)*(M)*L*T)))
ZS_config = vcat(ZS_config, ["buffer_len $bufferLen"])
interleaver_map = [mod(j,2) == 0 ? i-ϕ(i,Int64(j/2))[1] : ϕ(i,Int64((j-1)/2))[2]  for i in 0:T-1, j in 0:2*M*L*T-1]
interleaver_map = prod(string.(interleaver_map) .* " ", dims = 2)
ZS_config = vcat(ZS_config, ["interleaver_map"])
ZS_config = vcat(ZS_config, interleaver_map)
writedlm("ZS_config_file.txt", ZS_config)